<a href="https://colab.research.google.com/github/SebasHM12/Proyecto_Final/blob/main/PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias y conexión con drive

In [ ]:
#Instalar bibliotecas necesarias
!pip install numpy matplotlib opencv-python mediapipe
# Importar las bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import os
# Montar Google Drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow


#Etapa 1
#Preprocesamiento de imágenes


In [ ]:
# Definir funciones de procesamiento de imagen
# Aplicación de redimención
def resize_image(img, size=(224, 224)):
    """Redimensionar la imagen a un tamaño específico."""
    return cv2.resize(img, size, interpolation=cv2.INTER_CUBIC)
# Aplicación de filtro bilateral
def bilateral_filter(img, d=7, sigmaColor=31, sigmaSpace=31):
    """Aplicar un filtro bilateral para suavizar la imagen sin perder detalles."""
    return cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace)
# Aplicaón de contraste
def contrast(image, alpha=1.1, beta=2):
    """Mejorar el contraste de la imagen."""
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# Entradas y salidas
input_dirs = [
    "/content/drive/MyDrive/PT/Emotions/Happy",
    "/content/drive/MyDrive/PT/Emotions/Sad",
    "/content/drive/MyDrive/PT/Emotions/Neutral",
    "/content/drive/MyDrive/PT/Emotions/Surprise",
    "/content/drive/MyDrive/PT/Emotions/Fear",
    "/content/drive/MyDrive/PT/Emotions/Disgust",
    "/content/drive/MyDrive/PT/Emotions/Angry"
]
output_dir = "/content/drive/MyDrive/PT/Emotions_Dataset/"

# Asegura que el directorio de salida exista, si no, lo crea
os.makedirs(output_dir, exist_ok=True)

#The for loop should be at the same indentation level as the previous line
for input_dir in input_dirs:
    # Obtener el nombre de la carpeta
    folder_name = os.path.basename(input_dir)

    # Crear el directorio de salida específico para la carpeta
    folder_output_dir = os.path.join(output_dir, folder_name)
    os.makedirs(folder_output_dir, exist_ok=True)

    # Obtener la lista de archivos en el directorio de entrada
    files = os.listdir(input_dir)

    # Procesar cada archivo de imagen
    for idx, file in enumerate(files):
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            # Cargar la imagen
            img_path = os.path.join(input_dir, file)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

            if img is None:
                continue  # Si no se puede leer la imagen, pasar a la siguiente

            # Redimensionar la imagen a 224x224
            resized = resize_image(img, size=(224, 224))

            # Aplicar filtro bilateral
            smoothed = bilateral_filter(resized, d=1,
                                        sigmaColor=10, sigmaSpace=75)

            # Aplicar contraste
            contrasted = contrast(smoothed, alpha=1.2, beta=2)

            # Guardar la imagen procesada en el directorio de salida
            output_path = os.path.join(folder_output_dir, file)
            cv2.imwrite(output_path, contrasted)


print("Proceso completado.")


#Estapa 2
# Arquitectura y entrenamiento del modelo (ViT)

In [ ]:
# Instalación de la librería 'datasets' para trabajar con conjuntos de datos de Hugging Face
!pip install --upgrade datasets
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import cv2
import os
import torch
import seaborn as sns
from sklearn.model_selection import train_test_split
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import notebook_login
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer, AutoConfig
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomRotation,
    RandomResizedCrop,
    RandomHorizontalFlip,
    Resize,
    ToTensor,
    ColorJitter,
    RandomPerspective,
    RandomAffine,
    RandomVerticalFlip,
    RandomAdjustSharpness

)
from transformers import TrainerCallback
import itertools
import matplotlib.image as mpimg
from google.colab import drive
from torch.optim import AdamW
from transformers.optimization import get_scheduler

# Montar Google Drive
drive.mount('/content/drive')

# Cargar conjunto de datos
data_dir = '/content/drive/MyDrive/PT/Emotions_Dataset'

# Cargue el conjunto de datos usando el formato 'carpeta de imágenes'
my_dataset = load_dataset("imagefolder", data_dir=data_dir)

# Defina el punto de control del modelo
model_checkpoint ="motheecreator/vit-Facial-Expression-Recognition"

# Cargar el procesador de imágenes asociado al modelo preentrenado
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

# Obtenga la media, la norma y el tamaño de la imagen para normalizarla y cambiar su tamaño
image_mean, image_std = image_processor.image_mean, image_processor.image_std
size = image_processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)

# Define las transformaciones, para el entrenamiento y evaluación del modelo
# Las tranformaciones ayudan al modelo a ser mas robusto y tener más variedad de datos.
# Algunas transformaciones que se incluyen son: redimensión, rotaciones, converir a tensor y normalizar.
train_tf = Compose(
    [
        Resize((size, size)),
        RandomRotation(90),
        RandomAdjustSharpness(2),
        RandomHorizontalFlip(0.5),
        ToTensor(),
        normalize
    ]
)
# Definir las transformaciones para las imágenes de validación (solo redimensionado y normalización)
val_tf = Compose(
    [
        Resize((size, size)),
        ToTensor(),
        normalize
    ]
)

# Aplicar transformaciones a las imágenes de entrenamiento
def train_transforms(examples):
    examples['pixel_values'] = [train_tf(image.convert("RGB")) for image in examples['image']]
    return examples
# Aplicar transformaciones a las imágenes de validación
def val_transforms(examples):
    examples['pixel_values'] = [val_tf(image.convert("RGB")) for image in examples['image']]
    return examples

# Dividir el conjunto de datos en entrenamiento (90%) y validación (10%) usando splits
splits = my_dataset["train"].train_test_split(test_size=0.1)
train_data = splits['train']
val_data = splits['test']

# Aplicar las transformaciones definidas a los conjuntos de entrenamiento y validación
train_data.set_transform(train_transforms)
val_data.set_transform(val_transforms)

# Obtener nombres de las etiquetas del dataset
labels = my_dataset["train"].features["label"].names

# Crear mapeos de etiquetas de nombres a índices y viceversa
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

# Cargar la configuración del modelo
config = AutoConfig.from_pretrained(model_checkpoint)

# Configurar el dropout
config.hidden_dropout_prob = 0.2  # Porcentaje de Dropout en capas ocultas
config.attention_probs_dropout_prob = 0.4  # Porcentaje de Dropout en atención


# Cargar el modelo preentrenado para la clasificación de imágenes
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,  # Asignar los mapeos de etiquetas al modelo
    id2label=id2label,
    ignore_mismatched_sizes=True # Ignorar tamaños de pesos si hay discrepancias
)


# Crear un callback para almacenar las métricas de entrenamiento y observar como avanza el mismo
class TrainingMetricsCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.train_losses = [] # Lista para almacenar las pérdidas de entrenamiento
        self.val_losses = [] # Lista para almacenar las pérdidas de validación
        self.accuracies = [] # Lista para almacenar las precisiones de validación

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Guardar las métricas de entrenamiento cuando se registran para las curvas de aprendizaje
        if 'loss' in logs:
            self.train_losses.append(logs['loss'])
        if 'eval_loss' in logs:
            self.val_losses.append(logs['eval_loss'])
        if 'eval_accuracy' in logs:
            self.accuracies.append(logs['eval_accuracy'])

# Crear una instancia del callback para el entrenamiento
training_metrics_callback = TrainingMetricsCallback()

# Configurar argumentos de entrenamiento
args = TrainingArguments(
    output_dir="./resultados", # Directorio donde se guardarán los resultados
    logging_dir="./logs",      # Directorio donde se guardarán los registros de entrenamiento
    remove_unused_columns=False, # Mantener todas las columnas del dataset
    eval_strategy="steps",     # Estrategia de evaluación durante el entrenamiento
    save_strategy="steps",     # Estrategia de guardado del modelo durante el entrenamiento
    learning_rate=3e-05,       # Tasa de aprendizaje para el optimizador
    lr_scheduler_type="cosine",  # Tipo de scheduler de tasa de aprendizaje
    per_device_train_batch_size=32, # Tamaño de batch para entrenamiento
    gradient_accumulation_steps=8,  # Acumulación de gradientes para pasos
    per_device_eval_batch_size=32, # Tamaño de batch para evaluación
    weight_decay=0.1,              # Decaimiento de peso para el optimizador
    num_train_epochs= 16,           # Número de épocas de entrenamiento
    warmup_steps=1000,             # Pasos de calentamiento para el scheduler
    logging_steps=100,              # Registros de entrenamiento
    eval_steps=50,                 # Registros de evaluación
    load_best_model_at_end=True,   # Cargar el mejor modelo al finalizar
    metric_for_best_model="accuracy", # Métricas para el mejor modelo
    push_to_hub=False,             # No subir el modelo a Hugging Face
    report_to="none"               # No generar reportes
)


# Función para calcular las métricas de evaluación
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}
# Función collate para preparar los datos en batches
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([item['pixel_values'] for item in batch]),
        'labels': torch.tensor([item['label'] for item in batch])
    }


# Definir el optimizador y el scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, betas=(0.9, 0.999),
                  eps=1e-08)
scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=args.warmup_steps,
    num_training_steps=args.num_train_epochs * len(train_data)
)
# Definir el callback para la parada temprana, este ayudara a que el modelo no se sobreajuste
# La métrica que vigila es la de eval_accuracy, si esta no mejora despue de 2 veces el entrenamiento se detiene.
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=2, metric="eval_accuracy"):
        super().__init__()
        self.patience = patience  # Número de evaluaciones consecutivas sin mejora
        self.metric = metric
        self.best_metric = None
        self.num_bad_epochs = 0

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        current_metric = metrics.get(self.metric)
        if self.best_metric is None or current_metric > self.best_metric:
            self.best_metric = current_metric
            self.num_bad_epochs = 0
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            control.should_training_stop = True  # Indica que el entrenamiento debe detenerse

# Crear una instancia del callback para la parada temprana
early_stopping_callback = EarlyStoppingCallback(patience=2, metric="eval_accuracy")

# Entrenar el modelo
trainer = Trainer(
    model=model,  # Modelo a entrenar
    args=args,    # Argumentos de entrenamiento
    train_dataset=train_data,  # Conjunto de datos de entrenamiento
    eval_dataset=val_data,     # Conjunto de datos de validación
    tokenizer=image_processor,  # Procesador de imágenes
    compute_metrics=compute_metrics,  # Función para calcular métricas
    data_collator=collate_fn,  # Función para preparar datos en batches
    optimizers=(optimizer, scheduler),  # Optimizador y scheduler
    callbacks=[training_metrics_callback, early_stopping_callback]  # Añadir ambos callbacks
)


# Entrenar el modelo
trainer.train()

# Evaluar el modelo y guardar las métricas
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Graficar la curva de aprendizaje después del entrenamiento
plt.figure(figsize=(10, 5))

plt.plot(range(0, len(training_metrics_callback.train_losses)*args.eval_steps, args.eval_steps),
         training_metrics_callback.train_losses, label='Training Loss')

plt.plot(range(0, len(training_metrics_callback.val_losses)*args.eval_steps, args.eval_steps),
         training_metrics_callback.val_losses, label='Validation Loss')

plt.plot(range(0, len(training_metrics_callback.accuracies)*args.eval_steps, args.eval_steps),
         training_metrics_callback.accuracies, label='Validation Accuracy')

plt.xlabel('Steps')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.title('Learning Curves')
plt.show()

# Guardar modelo
# Guardar el modelo entrenado y el procesador de imágenes en Google Drive
output_dir = "/content/drive/MyDrive/PT/trained_model"
os.makedirs(output_dir, exist_ok=True)  # Crear el directorio si no existe
trainer.save_model(output_dir)  # Guardar el modelo entrenado
image_processor.save_pretrained(output_dir)  # Guardar el procesador de imágenes


 # Evaluación del modelo

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, classification_report
from transformers import AutoModelForImageClassification, AutoImageProcessor, Trainer
import pandas as pd

# Definir las rutas
dataset_path = "/content/drive/MyDrive/PT/Evaluacion"
model_path = '/content/drive/MyDrive/PT/trained_model'
output_dir = "/content/evaluation_results"
os.makedirs(output_dir, exist_ok=True)

# Cargar el procesador de imagen y las transformaciones
image_processor = AutoImageProcessor.from_pretrained(model_path)

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Ajustar al tamaño esperado por el modelo
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)  # Normalización basada en el procesador
])

# Cargar el conjunto de datos, transformaciones y división del conjunto
dataset = ImageFolder(root=dataset_path)
dataset.transform = transform
test_indices = np.arange(len(dataset))
test_dataset = torch.utils.data.Subset(dataset, test_indices)
# Crear DataLoader
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
# Cargar el modelo
model = AutoModelForImageClassification.from_pretrained(model_path)
# Definir el collator personalizado
class CustomCollator:
    def __call__(self, batch):
        images = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        return {'pixel_values': images, 'labels': labels}

data_collator = CustomCollator()


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    eval_dataset=test_dataset,
    tokenizer=image_processor
)

# Evaluar el modelo
eval_results = trainer.evaluate()
print("Evaluation Results:")
print(eval_results)

# Guardar resultados de evaluación
eval_results_path = os.path.join(output_dir, "eval_results.json")
with open(eval_results_path, "w") as f:
    f.write(str(eval_results))

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Calcular métricas
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
cm = confusion_matrix(labels, preds)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Función para graficar la matriz de confusión
def plot_confusion_matrix(cm, labels, output_path):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicción')
    plt.ylabel('Verdadero')
    plt.title('Matriz de Confusión')
    plt.savefig(output_path)
    plt.show()

emotions_labels = ['Enojado', 'Desagrado', 'Miedo', 'Feliz', 'Neutral', 'Triste', 'Sorpresa']

# Graficar y guardar la matriz de confusión
confusion_matrix_path = os.path.join(output_dir, "confusion_matrix.png")
plot_confusion_matrix(cm, emotions_labels, confusion_matrix_path)

# Función para graficar la distribución de clases
def plot_class_distribution(labels, preds, class_names, output_path):
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    sns.countplot(x=labels, ax=axs[0], palette='viridis')
    axs[0].set_title('Distribución de Clases Reales')
    axs[0].set_xticklabels(class_names, rotation=90)
    axs[0].set_ylabel('Número de Imágenes')
    axs[0].set_xlabel('Clases')

    sns.countplot(x=preds, ax=axs[1], palette='viridis')
    axs[1].set_title('Distribución de Clases Predichas')
    axs[1].set_xticklabels(class_names, rotation=90)
    axs[1].set_ylabel('Número de Imágenes')
    axs[1].set_xlabel('Clases')

    plt.tight_layout()
    plt.savefig(output_path)
    plt.show()

# Guardar gráfico de distribución de clases
class_distribution_path = os.path.join(output_dir, "class_distribution.png")
plot_class_distribution(labels, preds, emotions_labels, class_distribution_path)

# Crear y guardar reporte de clasificación
classification_report_str = classification_report(labels, preds, target_names=emotions_labels, digits=4)
print("Classification Report:")
print(classification_report_str)

classification_report_path = os.path.join(output_dir, "classification_report.txt")
with open(classification_report_path, "w") as f:
    f.write(classification_report_str)


# LIME

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Instalar las dependencias necesarias
!pip install lime torch transformers matplotlib
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
import os
import matplotlib.pyplot as plt

# Carga el extractor de características y el modelo
extractor = AutoFeatureExtractor.from_pretrained("/content/drive/MyDrive/PT/trained_model")
model = AutoModelForImageClassification.from_pretrained("/content/drive/MyDrive/PT/trained_model")
model.eval()

# Carpeta con las imágenes
carpeta_imagenes = "/content/drive/MyDrive/PT/LIME"

# Función para preprocesar una imagen
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = extractor(images=image, return_tensors="pt")
    image_array = inputs["pixel_values"].squeeze(0).permute(1, 2, 0).numpy()
    return image_array

# Define la función de predicción para LIME
def predict_fn(images):
    model_inputs = torch.tensor(images).permute(0, 3, 1, 2)
    with torch.no_grad():
        outputs = model(model_inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probabilities.numpy()

# Inicializa LIME
explainer = lime_image.LimeImageExplainer()

# Función para analizar y mostrar las imágenes y las explicaciones de LIME
def analyze_images(image_folder):
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    fig, axes = plt.subplots(len(image_files), 2, figsize=(10, 5 * len(image_files)))

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(image_folder, image_file)

        image_array = preprocess_image(image_path)

        # Explicar la predicción usando LIME
        explanation = explainer.explain_instance(
            image_array,
            predict_fn,
            top_labels=5,
            hide_color=0,
            num_samples=1000,
        )

        temp, mask = explanation.get_image_and_mask(
            label=explanation.top_labels[0],
            positive_only=True,
            num_features=5,
            hide_rest=False
        )

        # Mostrar la imagen original
        axes[i, 0].imshow(image_array / 2 + 0.5)
        axes[i, 0].set_title(f"Imagen Original: {image_file}")
        axes[i, 0].axis("off")

        # Mostrar la explicación de LIME
        axes[i, 1].imshow(mark_boundaries(temp / 2 + 0.5, mask))
        axes[i, 1].set_title(f"Explicación LIME: {image_file}")
        axes[i, 1].axis("off")

    plt.tight_layout()
    plt.show()

analyze_images(carpeta_imagenes)


Jaccard


In [ ]:
import torch
from lime import lime_image
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from sklearn.metrics import jaccard_score
import numpy as np
import matplotlib.pyplot as plt

# Carga el extractor de características y el modelo
extractor = AutoFeatureExtractor.from_pretrained("/content/drive/MyDrive/PT/trained_model")
model = AutoModelForImageClassification.from_pretrained("/content/drive/MyDrive/PT/trained_model")
model.eval()

# Función para preprocesar una imagen
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = extractor(images=image, return_tensors="pt")
    image_array = inputs["pixel_values"].squeeze(0).permute(1, 2, 0).numpy()
    return image_array

# Define la función de predicción para LIME
def predict_fn(images):
    model_inputs = torch.tensor(images).permute(0, 3, 1, 2)
    with torch.no_grad():
        outputs = model(model_inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probabilities.numpy()

# Inicializa LIME
explainer = lime_image.LimeImageExplainer()

image1_path = "/content/drive/MyDrive/PT/LIME/Sad.jpg"
image2_path = "/content/drive/MyDrive/PT/LIME/Surprice.jpg"

image1_array = preprocess_image(image1_path)
image2_array = preprocess_image(image2_path)

# Obtener explicaciones con LIME para ambas imágenes
explanation1 = explainer.explain_instance(
    image1_array,
    predict_fn,
    top_labels=5,
    hide_color=0,
    num_samples=1000
)

explanation2 = explainer.explain_instance(
    image2_array,
    predict_fn,
    top_labels=5,
    hide_color=0,
    num_samples=1000
)

# Obtener las máscaras de ambas imágenes
_, mask_image1 = explanation1.get_image_and_mask(
    label=explanation1.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)

_, mask_image2 = explanation2.get_image_and_mask(
    label=explanation2.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)

# Función para calcular el índice de Jaccard
def calculate_jaccard(mask1, mask2):
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()
    return jaccard_score(mask1_flat, mask2_flat, average='binary')

# Calcular el índice de Jaccard
jaccard_index = calculate_jaccard(mask_image1, mask_image2)

print(f"Índice de Jaccard entre las dos imágenes: {jaccard_index:.4f}")

# Visualizar las imágenes y sus máscaras
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Imagen 1
axes[0, 0].imshow(image1_array / 2 + 0.5)
axes[0, 0].set_title("Imagen 1 Original")
axes[0, 1].imshow(mask_image1)
axes[0, 1].set_title("Máscara LIME Imagen 1")

# Imagen 2
axes[1, 0].imshow(image2_array / 2 + 0.5)
axes[1, 0].set_title("Imagen 2 Original")
axes[1, 1].imshow(mask_image2)
axes[1, 1].set_title("Máscara LIME Imagen 2")

plt.tight_layout()
plt.show()


# Etapa 3
# Aplicacion web

In [41]:
# Instalar librerias
!pip install streamlit
!pip install --upgrade streamlit
!pip install streamlit-option-menu



In [42]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
from torchvision import transforms
from streamlit_option_menu import option_menu
import os

# Cargar el modelo y el procesador de imágenes
model_path = '/content/drive/MyDrive/PT/trained_model'
model = AutoModelForImageClassification.from_pretrained(model_path)
image_processor = AutoImageProcessor.from_pretrained(model_path)

# Definir las transformaciones de la imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

# Función para predecir la emoción en una imagen
def predict_emotion(image):
    image = transform(image).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        top_probs, top_labels = torch.topk(probs, probs.size(1), dim=1)
    predictions = [{'score': score.item(), 'label': model.config.id2label[label.item()]}
                   for score, label in zip(top_probs[0], top_labels[0])]
    return predictions

# Inicializar el estado de la aplicación
if 'menu_option' not in st.session_state:
    st.session_state.menu_option = "Principal"
if 'selected_image' not in st.session_state:
    st.session_state.selected_image = None
if 'uploaded_image' not in st.session_state:
    st.session_state.uploaded_image = None  # Variable para almacenar la imagen cargada

# Interfaz de Streamlit
st.set_page_config(page_title="Detección de emociones", layout="centered")

with st.sidebar:
    st.header("Bienvenid@ a mi aplicación.")
    st.write("""
    Esta es una aplicación para la detección de emociones, diseñada para identificar las diferentes
    emociones que una persona puede expresar. Las imágenes no solo capturan momentos, también son
    poderosas fuentes de emociones. Un solo gesto puede revelar una historia, y esta herramienta está
    aquí para ayudarte a descifrarla.
    ¡Explora y descubre las emociones a través de tus imágenes, porque a veces una imagen dice más que mil palabras!""")
    st.write("---")
    st.write("")
    st.write("")
    st.write("")
    st.write("Si necesitas ayuda presiona el boton😀")
    if st.sidebar.button("Ayuda"):
      st.sidebar.markdown("Video de ayuda: [Haz clic aquí](https://www.youtube.com/watch?v=zeS2FlxF_0s&t=1702s)")

# Agregar logos en la parte superior
logo1 = Image.open('/content/drive/MyDrive/PT/Imagenes_Streamlit/logo.png')
logo2 = Image.open('/content/drive/MyDrive/PT/Imagenes_Streamlit/UAM.png')

# Contenedor para los logos
logo_container = st.container()
with logo_container:
    col1, col2, col3 = st.columns([1, 2, 1])
    with col1:
        st.image(logo1, width=100)
    with col2:
        st.write("")
    with col3:
        st.image(logo2, width=400)

# Menú de opciones en la parte superior
st.session_state.menu_option = option_menu("___________________Menu___________________",
                                             [ "Conoce", "Principal",  "Créditos"],
                                             icons=[ 'question-circle','house',  'person-fill'],
                                             menu_icon='three-dots',
                                             default_index=1,
                                             orientation='horizontal')

# Define un diccionario de emociones y sus emojis
emotion_emojis = {
    "Feliz": "😊",
    "Triste": "😢",
    "Sorpresa": "😮",
    "Enojo": "😡",
    "Miedo": "😱",
    "Desagrado": "😖",
    "Neutral": "😐"
}

#Opcion de principal y componentes
if st.session_state.menu_option == "Principal":

    st.header("Detección de Emociones en Imágenes")
    st.write("---")
    st.write("😲 ¡Descubre la emoción que tu imagen puede revelar! 😄📸")
    st.write("""Esta aplicación está diseñada para analizar imágenes que muestren un solo rostro.
    Asegúrate de que la imagen destaque una única cara para obtener los mejores resultados.""")

    # Opción para cargar una imagen desde la PC
    st.subheader("Cargar imagen desde mi PC")
    uploaded_file = st.file_uploader("", type=["jpg", "jpeg", "png"], label_visibility="collapsed")

    col1, col2 = st.columns([2, 1])
    col2.subheader("Tomar una foto")
    camera_photo = col2.camera_input(" ", label_visibility="collapsed")

    # Revisar si hay un archivo subido o una foto tomada
    image = None

    if uploaded_file is not None:
        try:
            image = Image.open(uploaded_file).convert('RGB')
            col2.success("La imagen fue cargada correctamente.")
            st.session_state.uploaded_image = image  # Almacena la imagen cargada
            st.session_state.selected_image = None  # Limpiar selección de imagen de prueba
        except Exception:
            col2.error("⚠️Error al cargar la imagen 😥. Por favor asegúrate de que el archivo sea válido 🧐.")
    elif camera_photo is not None:
        try:
            image = Image.open(camera_photo).convert('RGB')
            col2.success("La foto fue tomada correctamente.")
            st.session_state.uploaded_image = image  # Almacena la foto tomada
            st.session_state.selected_image = None  # Limpiar selección de imagen de prueba
        except Exception:
            col2.error("Error al tomar la foto.")

    # Galería de imágenes de prueba
    st.write("### Imágenes para probar")
    test_images_folder = '/content/drive/MyDrive/PT/Imagenes_Streamlit/Prueba'
    image_files = [f for f in os.listdir(test_images_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

    num_images_per_row = 5
    num_rows = (len(image_files) + num_images_per_row - 1) // num_images_per_row

    for row in range(num_rows):
        cols = st.columns(num_images_per_row)
        for col in range(num_images_per_row):
            idx = row * num_images_per_row + col
            if idx < len(image_files):
                img_file = image_files[idx]
                img_path = os.path.join(test_images_folder, img_file)
                test_image = Image.open(img_path).convert('RGB')
                with cols[col]:
                    st.image(test_image, width=100)
                    if st.button(f"Probar", key=img_file):
                        st.session_state.selected_image = img_file
                        st.session_state.uploaded_image = None

    # Si hay una imagen disponible (cargada o seleccionada de la galería), mostrarla a la derecha y las probabilidades a la izquierda
    if image is not None or st.session_state.selected_image is not None:
        image_to_classify = image if image is not None else Image.open(os.path.join(test_images_folder, st.session_state.selected_image)).convert('RGB')
        col2.image(image_to_classify, caption='Imagen Cargada', width=300)
        with col1:
            st.write("Clasificando...")
            predictions = predict_emotion(image_to_classify)

            # Muestra la emoción detectada con su emoji
            detected_emotion = predictions[0]['label']
            emoji = emotion_emojis.get(detected_emotion, "❓")  # Usa un emoji de pregunta si no se encuentra la emoción
            st.write("Emoción detectada:")
            st.write(f"**{detected_emotion} {emoji}**")

            st.write("Probabilidades:")
            for pred in predictions:
                st.progress(pred['score'])
                emoji = emotion_emojis.get(pred['label'], "❓")  # Usa un emoji de pregunta si no se encuentra la emoción
                st.write(f"{pred['label']} {emoji}: {pred['score']:.2f}")

# Opción de conoce y sus componentes
elif st.session_state.menu_option == "Conoce":
    st.header("Transformadores Visuales y Reconocimiento de Emociones")
    st.write("### Transformadores Visuales (Vision Transformers - ViT)")
    st.write("""Los transformadores visuales (ViT) son un avance reciente en el campo de la visión por computadora.
    Basados en la arquitectura de transformadores utilizada originalmente para procesamiento de lenguaje natural,
    los ViT dividen las imágenes en 'parches' y procesan estos parches de manera similar a cómo los
    transformadores manejan secuencias de texto. Esto permite que el modelo aprenda relaciones espaciales
    complejas entre diferentes partes de la imagen, resultando en una comprensión más profunda del contenido.""")
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.image('/content/drive/MyDrive/PT/Imagenes_Streamlit/conoce1.png',
                 caption='Arquitectura de Vision Transformers',
                 width=400)

    # Segunda sección: Aplicaciones de los Modelos de Vision Transformers
    st.write("---")
    col1, col2 = st.columns([2, 1])
    with col1:
        st.write("### Aplicaciones de los Modelos de Vision Transformers")
        st.write("""El uso de Vision Transformers en el reconocimiento de emociones es
        particularmente beneficioso, ya que pueden capturar detalles sutiles en las
        expresiones faciales humanas. Esto les permite identificar emociones con mayor
        precisión en situaciones complejas, como en imágenes con múltiples personas o
        con expresiones faciales mezcladas. Su capacidad para aprender patrones complejos
        a partir de grandes cantidades de datos facilita su uso en tareas como el análisis
        de emociones en tiempo real.""")
    with col2:
        st.image('/content/drive/MyDrive/PT/Imagenes_Streamlit/aplicacion.jpg',
                 caption='ViT en acción detectando emociones',
                 width=300)

    # Tercera sección: ViT para el Reconocimiento de Emociones
    st.write("---")
    col1, col2 = st.columns([1, 2])
    with col2:
        st.write("### Modelo ViT para el Reconocimiento de Emociones")
    col1, spacer, col2 = st.columns([1, 0.5, 2])
    with col1:
        st.image('/content/drive/MyDrive/PT/Imagenes_Streamlit/conoce2.png',
                 caption='Modelo ViT para emociones',
                 width=250)
    with col2:
        st.write("""En nuestra aplicación, utilizamos un modelo de Vision Transformers
        entrenado específicamente para la clasificación de emociones en imágenes.
        Este modelo ha sido ajustado con un conjunto de datos robusto que incluye
        expresiones faciales diversas. Gracias a su arquitectura, el modelo puede
        distinguir entre emociones como felicidad, tristeza, sorpresa y más con
        alta precisión, lo que lo convierte en una herramienta valiosa para
        aplicaciones en psicología, atención al cliente, y más.""")

elif st.session_state.menu_option == "Créditos":
    st.header("Créditos")
    st.write("Desarrollado por: Sebastian Hernández Mejía")
    st.write("Alumno de la carrera Licenciatura en Ingeniería en Computación")
    st.write("""Basado en 'Mothercreater/vit-Facial-Expression-Recognition', este modelo fue ajustado finamente con un conjunto de datos
    adicional que incluye imágenes de rostros en diversas expresiones emocionales.""")
    st.write("Datos: Se utilizó un conjunto de imágenes de AffectNet para el entrenamiento.")
    st.write("Agradecimientos especiales a mis asesores:")
    st.write("- Dra. Silvia Beatriz González Brambila")
    st.write("- M. en C. Josué Figueroa González")


Overwriting app.py


In [ ]:
# Levantar la aplicación
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.211.185:8501

your url is: https://young-keys-refuse.loca.lt
2024-09-23 03:23:13.548489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 03:23:13.577675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 03:23:13.586683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 03:23:13.607546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU inst